### **Model Danych Pythona**

Model danych Pythona opisuje interfejs API, którego możemy używać do tworzenia własnych obiektów działających dobrze z najbardziej idiomatycznymi funkcjonalnościami tego języka. Model danych możemy uważać za opis Pythona jako platformy a jego zadaniem jest formalizacja interfejsu bloków konstrukcyjnych samego języka, takich jak sekwencje, iteratory, klasy, menedżery kontekstu itp.

### **Pythoniczna Talia Kart**

In [66]:
import collections

Card = collections.namedtuple("Card", ["rank", "suit"])


class FrenchDeck:
    ranks = list(str(s) for s in range(2, 11)) + list("JQKA")
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = list(Card(rank, suit) for rank in self.ranks for suit in self.suits)

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [67]:
deck = FrenchDeck()

In [68]:
# Tak jak wszystkie kolekcje Pythona, FrenchDeck odpowiada na funkcję len()
len(deck)

52

In [69]:
# Odczytywanie kart zapewnia metoda __getitem__
deck[0]

Card(rank='2', suit='spades')

In [70]:
# Do wyboru losowej karty możemy użyć modułu random
from random import choice

choice(deck)

Card(rank='9', suit='spades')

In [71]:
# Metoda __getitem__ odwołuje się do operatora [] i obsługuje wycinanie
deck[3:13:2]

[Card(rank='2', suit='hearts'),
 Card(rank='3', suit='diamonds'),
 Card(rank='3', suit='hearts'),
 Card(rank='4', suit='diamonds'),
 Card(rank='4', suit='hearts')]

In [72]:
# Iterowanie również jest możliwe, dzięki __getitem__
for card in deck[:3]:
    print(card)

Card(rank='2', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='clubs')


In [73]:
# Iteracja jest często niejawna. Jeśli kolekcja nie ma metody __contains__,
# Python przeszuka kolekcję od początku do końca
Card("Q", "hearts") in deck

True

In [74]:
# Sortowanie kart jest bardziej skomplikowane, ponieważ powinniśmy zdefiniować
# porządek kart. Najcenniejsze są asy, a następnie kolor.
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)


def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]


for card in sorted(deck[:5], key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='spades')


Dzięki implementacji metod specjalnych `__len__` oraz `__getitem__`, klasa `FrenchDeck` zachowuje się jak standardowa sekwencja Pythona, pozwalająca na korzystanie z podstawowych funkcjonalności języka oraz z biblioteki standardowej. 

### **Sposoby Używania Metod Specjalnych**

Najważniejszą cechą metod specjalnych jest to, że mają być wywoływane przez interpreter a nie przez programistów. Jeżeli `my_object` jest wystąpieniem klasy zdefiniowanej przez użytkownika wtedy w przypadku wywołania `len(my_object)`, Python wywoła zaimplementową metodę `__len__`. Jednakże interpreter używa skrótu w przypadku typów wbudowanych, takich jak `list`, `str` lub rozszerzeń, takich jak tablice NumPy. Kolekcje Pythona o zmiennym rozmiarze napisane w C zawierają strukturę o nazwie `PyVarObject`, która zawiera pole `ob_size` przechowujące liczbę elementów kolekcji. Tak więc, jeśli instancja jest wystąpieniem jednego z typów wbudowanych to wywołanie `len(my_object)` odczytuje pole ze struktury co jest znacznie szybsze niż wywołanie metody.

Najczęściej wywoływanie metod specjalnych odbywa się niejawnie. Na przykład instrukcja `for i in x:` w rzeczywistości powoduje wywołanie funkcji `iter(x)`, która z kolei może wywołać metodę `x.__iter__()` jeśli jest ona dostępna, lub użyć `x.__getitem__()` i iterować po obiekcie zaczynając od zera. 

### **Reprezentacja Tekstowa**

Metoda specjalna `__repr__` jest wywoływana przez wbudowaną funkcję `repr()` aby otrzymać reprezentację tekstową obiektu do inspekcji. Łańcuch znaków zwracany przez `__repr__` powinien być jednoznaczny i o ile to możliwe odpowiadać kodowi źródłowemu koniecznemu do ponownego utworzenia reprezentowanego obiektu.

Dla kontrastu metoda `__str__` jest wywoływana przez wbudowaną funkcję `str()` i niejawnie używana w funkcji `print()`. Metoda ta powinna zwracać łańcuch znaków odpowiedni dla użytkowników końcowych. 

W przypadku implementacji tylko jednej z tych metod specjalnych, lepiej wybrać `__repr__`, ponieważ gdy nie ma dostępnej niestandardowej metody `__str__`, Python wywoła `__repr__` jako metodę rezerwową.

### **Wartość Logiczna Typu Niestandardowego**

In [81]:
import math


class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(self.x or self.y)

    def __repr__(self):
        return f"{type(self).__name__}({self.x!r}, {self.y!r})"

In [82]:
v = Vector(3, 4)
abs(v), bool(v), repr(v)

(5.0, True, 'Vector(3, 4)')

Chociaż Python ma typ `bool`, akceptuje dowolny obiekt w kontekstach logicznych, takich jak wyrażenia kontrolujące instrukcje `if` lub `while` albo jako operandy operatorów `and`, `or` i `not`. Aby wyznaczyć czy wartość jest truthy (prawdziwa) lub falsy (fałszywa), Python stosuje `bool(x)`, co zawsze zwraca `True` lub `False`.

Domyślnie wystąpienia klas definiowanych przez użytkownika są uważane za truthy pod warunkiem, że nie implementują metod `__bool__` ani `__len__`. Jeśli metoda `__bool__` nie jest zaimplementowana, Python spróbuje wywołać metodę `__len__` i wynikiem `bool(x)` jest wynik tejże metody.

### **API Kolekcji**

<p align="center">
  <img src="ABC.png"/>
</p>

Każda z najwyższych klas ABC zawiera pojedynczą metodę specjalną. Klasa ABC `Collection` unifikuje trzy kluczowe interfejsy, które powinna implementować każda kolekcja:
- `Iterable` - w celu obsługi pętli `for`, rozpakowywania i innych form iteracji.
- `Sized` - w celu wsparcia wbudowanej funkcji `len()`.
- `Container` - dla obsługi operatora `in`.

Python nie wymaga od konkretnych klas aby wywodziły się z którejkolwiek z tych klas ABC. Przykładowo dowolna klasa implementująca `__len__` spełnia wymagania interfejsu `Sized`.

Trzy bardzo ważne specjalizacje `Collection` to:
- `Sequence` - formalizuje interfejs wbudowanych funkcji, takich jak `list` i `str`.
- `Mapping` - implementowane przez `dict`, `defaultdict` itp.
- `Set` - interfejs typów wbudowanych `set` i `frozenset`. 

Tylko `Sequence` jest `Reversible`, gdyż sekwencje wspierają dowolne porządkowanie swojej zawartości, czego nie robią słowniki i zbiory.